In [1]:
# ================================================
# Project: Machine Learning in Business
# Goal:
#   Pick the best region for a new oil well to maximize profit
#   while controlling risk (≤ 2.5% probability of loss).
#
# Tasks:
#   1) Load/prepare data for three regions (features + product).
#   2) Train Linear Regression (as required) per region with 75/25 split.
#   3) Evaluate each model (RMSE, mean predicted reserves).
#   4) Compute economic thresholds (break-even per-well reserves).
#   5) Write a profit function:
#        - Explore 500 points, choose top 200 by prediction, compute profit.
#   6) Use Bootstrapping (1000 reps) on each region’s validation predictions
#      to estimate distribution of profit; compute:
#        - Average profit
#        - 95% confidence interval
#        - Risk of loss (P(profit < 0))
#   7) Recommend a region if risk < 2.5%, otherwise reject.
#
# Why these choices:
#   - LinearRegression: specified by business rules (predictability/interpretability).
#   - 75/25 split: instruction requires it; validation set is held-out for fair testing.
#   - Top-200 out of 500: simulates real selection with budget constraints.
#   - Bootstrapping: quantifies uncertainty in profit to make risk-aware decisions.
# ================================================

In [ ]:
# Core libraries
import pandas as pd
import numpy as np

# Modeling
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# Reproducibility
RANDOM_STATE = 12345
rng = np.random.RandomState(RANDOM_STATE)

# Small helper: RMSE
def rmse(y_true, y_pred):
    #Root Mean Squared Error — scale-sensitive error metric.
    #Chosen because 'product' is a continuous target and RMSE is standard for regression.
    return np.sqrt(mean_squared_error(y_true, y_pred))


In [3]:
# Why: Keep raw region data separate to avoid accidental mixing.
# Paths are from the task statement (ensure files exist in /datasets/).
region_paths = {
    "region_0": "/datasets/geo_data_0.csv",
    "region_1": "/datasets/geo_data_1.csv",
    "region_2": "/datasets/geo_data_2.csv",
}

regions = {name: pd.read_csv(path) for name, path in region_paths.items()}

# Quick structure check (no heavy EDA per task constraints)
for name, df in regions.items():
    print(f"{name}: shape={df.shape}, columns={list(df.columns)}")
    assert "product" in df.columns, "Target column 'product' must exist"


region_0: shape=(100000, 5), columns=['id', 'f0', 'f1', 'f2', 'product']
region_1: shape=(100000, 5), columns=['id', 'f0', 'f1', 'f2', 'product']
region_2: shape=(100000, 5), columns=['id', 'f0', 'f1', 'f2', 'product']


In [4]:
# Why: Encapsulate repeated logic into a function to avoid duplication
#      and to keep notebook clean and auditable.

def train_validate_region(df, region_name, random_state=RANDOM_STATE):
    #Split 75/25, train LinearRegression, evaluate on validation.
    #Returns a dict with model, predictions, metrics, and the validation frame.
    # Separate features/target (LinearRegression needs numeric features)
    X = df.drop(columns=["product", "id"], errors="ignore")
    y = df["product"]

    # 75/25 split as required
    X_train, X_valid, y_train, y_valid = train_test_split(
        X, y, test_size=0.25, random_state=random_state
    )

    # Linear model per business constraints
    model = LinearRegression()
    model.fit(X_train, y_train)

    # Predict on validation for fair evaluation
    y_pred = model.predict(X_valid)

    # Metrics required by the spec
    region_rmse = rmse(y_valid, y_pred)
    mean_pred = y_pred.mean()

    # Keep a tidy validation frame for later profit simulation
    valid_df = X_valid.copy()
    valid_df["product"] = y_valid.values
    valid_df["prediction"] = y_pred

    print(f"[{region_name}] mean(prediction)={mean_pred:.2f}, RMSE={region_rmse:.2f}")

    return {
        "model": model,
        "X_train": X_train, "X_valid": X_valid,
        "y_train": y_train, "y_valid": y_valid,
        "y_pred": y_pred,
        "rmse": region_rmse,
        "mean_pred": mean_pred,
        "valid_df": valid_df
    }

region_results = {}
for name, df in regions.items():
    region_results[name] = train_validate_region(df, name)


[region_0] mean(prediction)=92.59, RMSE=37.58
[region_1] mean(prediction)=68.73, RMSE=0.89
[region_2] mean(prediction)=94.97, RMSE=40.03


In [5]:
# Business constants from the brief
BUDGET = 100_000_000            # $100M total for 200 wells
PRICE_PER_BARREL = 4.5          # $/barrel
PRICE_PER_UNIT = 4500           # $ per 'product' unit (thousand barrels)
WELLS_TO_DEVELOP = 200
POINTS_TO_EXPLORE = 500

# Break-even reserves per well (in 'product' units = thousand barrels)
# Why: development cost per well = BUDGET / WELLS_TO_DEVELOP.
#      We break even when revenue = cost → required product = cost / revenue per unit.
cost_per_well = BUDGET / WELLS_TO_DEVELOP
break_even_product = cost_per_well / PRICE_PER_UNIT

print(f"Break-even reserves per developed well (thousand barrels): {break_even_product:.2f}")

# Compare with average 'product' per entire region (rough sense-check, not a decision rule)
for name, df in regions.items():
    print(f"[{name}] average true product: {df['product'].mean():.2f}")


Break-even reserves per developed well (thousand barrels): 111.11
[region_0] average true product: 92.50
[region_1] average true product: 68.83
[region_2] average true product: 95.00


In [6]:
def profit_from_selection(sample_df, n_select=WELLS_TO_DEVELOP,
                          price_per_unit=PRICE_PER_UNIT, budget=BUDGET):
    # Compute profit for a chosen set of wells.
    # Assumes `sample_df` contains columns: ['product', 'prediction'].
    #Strategy: select top n_select wells by predicted reserves, then sum *true* product.
    # Sort by predicted reserves descending, pick top N
    chosen = sample_df.sort_values(by="prediction", ascending=False).head(n_select)
    revenue = chosen["product"].sum() * price_per_unit
    return revenue - budget


def simulate_profit_once(valid_df, n_explore=POINTS_TO_EXPLORE, rng=rng):
    # Simulate the exploration campaign:
    # 1) Explore n_explore wells sampled (with replacement) from validation set.
    # 2) Select top 200 by prediction and compute realized profit from true product.
    # Why: mirrors the business process and bootstrapping strategy.
    # Sample with replacement to reflect sampling variability
    idx = rng.choice(valid_df.index, size=n_explore, replace=True)
    sample = valid_df.loc[idx]
    return profit_from_selection(sample)


def bootstrap_profit(valid_df, n_boot=1000, rng=rng):
    # Run many campaign simulations to estimate the distribution of profit.
    profits = [simulate_profit_once(valid_df, rng=rng) for _ in range(n_boot)]
    profits = pd.Series(profits)
    avg = profits.mean()
    ci_low, ci_high = profits.quantile([0.025, 0.975])
    risk = (profits < 0).mean()  # probability of loss
    return profits, avg, (ci_low, ci_high), risk


In [7]:
summary_rows = []

for name, res in region_results.items():
    valid_df = res["valid_df"]

    profits, avg_profit, (ci_low, ci_high), risk = bootstrap_profit(valid_df, n_boot=1000, rng=rng)

    row = {
        "region": name,
        "rmse": res["rmse"],
        "mean_pred": res["mean_pred"],
        "avg_profit": avg_profit,
        "ci_low": ci_low,
        "ci_high": ci_high,
        "risk_loss": risk
    }
    summary_rows.append(row)

summary = pd.DataFrame(summary_rows).sort_values(by="avg_profit", ascending=False).reset_index(drop=True)
summary


,region,rmse,mean_pred,avg_profit,ci_low,ci_high,risk_loss
0,region_1,0.893099,68.728547,4.611558e+06,7.805081e+05,8.629521e+06,0.007
1,region_0,37.579422,92.592568,3.961650e+06,-1.112155e+06,9.097669e+06,0.069
2,region_2,40.029709,94.965046,3.929505e+06,-1.122276e+06,9.345629e+06,0.065


In [8]:
# Business rule: risk of losses must be < 2.5%
safe = summary[summary["risk_loss"] < 0.025].copy()

if len(safe) == 0:
    print("No region meets the risk criterion (loss risk < 2.5%). Do not proceed.")
else:
    best = safe.sort_values(by="avg_profit", ascending=False).iloc[0]
    print("Recommended region:")
    print(f"  - {best['region']}")
    print(f"  - Avg profit: ${best['avg_profit']:,.0f}")
    print(f"  - 95% CI: [${best['ci_low']:,.0f}, ${best['ci_high']:,.0f}]")
    print(f"  - Loss risk: {best['risk_loss']*100:.2f}%")

print("\nFull summary:")
display(summary.style.format({
    "rmse": "{:.2f}",
    "mean_pred": "{:.2f}",
    "avg_profit": "${:,.0f}",
    "ci_low": "${:,.0f}",
    "ci_high": "${:,.0f}",
    "risk_loss": "{:.2%}",
}))

Recommended region:
  - region_1
  - Avg profit: $4,611,558
  - 95% CI: [$780,508, $8,629,521]
  - Loss risk: 0.70%

Full summary:


,region,rmse,mean_pred,avg_profit,ci_low,ci_high,risk_loss
0,region_1,0.89,68.73,"$4,611,558","$780,508","$8,629,521",0.70%
1,region_0,37.58,92.59,"$3,961,650","$-1,112,155","$9,097,669",6.90%
2,region_2,40.03,94.97,"$3,929,505","$-1,122,276","$9,345,629",6.50%


In [9]:
# Results Discussion

# Model Evaluation
# The Linear Regression models were trained separately for each region, and their predictive accuracy was measured using RMSE.

# | Region | RMSE | Mean Predicted Product |
# |---------|------|------------------------|
# | Region 0 | 37.58 | 92.59 |
# | **Region 1** | **0.89** | **68.73** |
# | Region 2 | 40.03 | 94.97 |

# Interpretation:
# - Region 1’s RMSE (0.89) is dramatically lower than the others, indicating an almost perfect linear relationship between the features and the target variable.  
# - Regions 0 and 2 show significantly higher RMSE (~40), suggesting greater variance and less reliable predictions.

# This tells us that the geological data in Region 1 is much more predictable, and the model can identify profitable wells with higher confidence.

# ---

# Break-Even Analysis
# The company’s development cost per well implies a break-even reserve volume of 111.11 thousand barrels.  
# Average reserves in each region are:

# | Region | Mean Actual Product |
# |---------|--------------------|
# | Region 0 | 92.50 |
# | Region 1 | 68.83 |
# | Region 2 | 95.00 |

# Interpretation:
# While all regions’ average reserves fall below the break-even threshold, the goal is to select only the top 200 of 500 wells, not the average ones.  
# Hence, even though Region 1’s overall average is smaller, its high model accuracy allows it to isolate wells above the profitable cutoff more effectively.

# ---

# Bootstrapping Profit Simulation
# Using 1,000 bootstrap iterations for each region (simulating different exploration campaigns):

# | Region | Avg Profit | 95% Confidence Interval | Risk of Loss |
# |---------|-------------|--------------------------|--------------|
# | **Region 1** | **\$4.61 M** | \$0.78 M – \$8.63 M | **0.7 %** |
# | Region 0 | \$3.96 M | −\$1.11 M – \$9.10 M | 6.9 % |
# | Region 2 | \$3.93 M | −\$1.12 M – \$9.35 M | 6.5 % |

# Interpretation:
# - Region 1 not only yields the highest mean profit but also has the smallest variability and risk.  
# - The confidence interval for Region 1 is entirely positive, while those for Regions 0 and 2 extend below zero, meaning there’s a tangible chance of financial loss in those areas.  
# - Risk of loss in Region 1 (0.7 %) is well below the 2.5 % threshold specified by management.

# ---

# Conclusion
# Region 1 is the best choice for developing new oil wells.  
# It meets all business and statistical criteria:

# - The Linear Regression model performs best (lowest RMSE, strongest predictability).  
# - The expected profit is highest among all regions.  
# - The risk of losses is minimal (0.7 %, comfortably below the 2.5 % limit).  
# - The confidence interval indicates **consistent, positive returns**.

# Therefore, based on both quantitative and business risk analysis, OilyGiant should proceed with well development in Region 1.
